### Logistic Regression Model

Want $ 0 \le h_\theta(x) \le 1 $

$h_\theta(x) = g(\theta^{T}x)$

**Sigmoid function/Logistic function**: $g(z) = \frac{1}{1+e^{-z}}$

**Hypothesis**: $h_\theta(x) = \frac{1}{1+e^{-\Theta^{T}x}}$

**Cost function**: $J(\theta) = \frac{1}{m}\sum_{i=1}^{m}Cost(h_\theta(x^{(i)}), y^{(i)}) $


$Cost(h_\theta(x),y) = $ <br>
$-log(h_\theta(x)) $&nbsp;&nbsp;&nbsp;if y = 1 <br>
$-log(1 - h_\theta(x))$&nbsp;&nbsp;&nbsp; if y = 0

$J(\theta) = -\frac{1}{m}[\sum_{i=1}^{m}y^{(i)}log(h_\theta(x^{(i)}))+(1-y^{(i)})log (1-h_\theta(x^{(i)}))]$

**Gradient descent**: <br>
Repeat { <br>
&nbsp;&nbsp;&nbsp;&nbsp;$\theta_j := \theta_j - \alpha\frac{1}{m}\sum_{i=1}^{m}(h_\theta(x^{(i)})-y^{(i)})x_j^{(i)}$<br>
}

### Regularization
**Cost function**: $J(\theta) = -\frac{1}{m}[\sum_{i=1}^{m}y^{(i)}log(h_\theta(x^{(i)}))+(1-y^{(i)})log (1-h_\theta(x^{(i)}))] + \frac{\lambda}{2m}\sum_{j=1}^{n}\theta_j^{2}$

**Gradient descent**: <br>
Repeat { <br>
&nbsp;&nbsp;&nbsp;&nbsp;$\theta_0 := \theta_0 - \alpha\frac{1}{m}\sum_{i=1}^{m}(h_\theta(x^{(i)})-y^{(i)})x_0^{(i)}$<br>
&nbsp;&nbsp;&nbsp;&nbsp;$\theta_j := \theta_j - [\alpha\frac{1}{m}\sum_{i=1}^{m}(h_\theta(x^{(i)})-y^{(i)})x_j^{(i)} - \frac{\lambda}{m}\theta_j]$<br>
}
